In [ ]:
def sgn(p):
    '''
    Give the signature of a permutation

    Parameters:
    -----------
    p: the permutation (a list)

    Return:
    ------
    1 if the number of exchange is an even number
    -1 if the number is an odd number
    '''

    if len(p) == 1:
        return True

    trans = 0

    for i in range(0, len(p)):
        j = i + 1

        for j in range(j, len(p)):
            if p[i] > p[j]:
                trans = trans + 1

    if (trans % 2) == 0:
        return 1
    else:
        return -1

In [ ]:
def OptimizeProblem(FIMSet):
    
    m = ConcreteModel()
    
    m.NoRes = Set(initialize=range(len(FIMSet)))
    m.DimFIM = Set(initialize=range(len(FIMSet[0][0])))
    
    m.y = Var(m.NoRes, initialize=0.1, bounds=(0,1), within = NonNegativeReals)

    def FIM_assign(m,y,d,e):
        return FIMSet[y][d,e]
    
    m.FIM = Param(m.NoRes, m.DimFIM, m.DimFIM, initialize=FIM_assign)
    
    def totalFIM(m,d,e):
        return sum(m.y[y] * m.FIM[y,d,e] for y in m.NoRes)
    
    m.TotalFIM = Var(m.DimFIM, m.DimFIM, initialize=totalFIM)
    
    def totalFIM_const(m,d,e):
        return m.TotalFIM[d,e] == sum(m.y[y] * m.FIM[y,d,e] for y in m.NoRes)
    m.TotalFIM_con = Constraint(m.DimFIM, m.DimFIM, rule=totalFIM_const)
    
    def ComputeTrace(m):
        sum_x = 0  
        for j in m.DimFIM:
            for d in m.DimFIM:
                if d==j:
                    sum_x += m.TotalFIM[j,d]
        return sum_x
    
    m.Obj = Objective(rule=ComputeTrace, sense=maximize)    
    
    def Sum1Const(m):
        return sum(m.y[y] for y in m.NoRes)==1 
    m.sum1 = Constraint(rule=Sum1Const)
    
    return m 
    
    


In [ ]:
def OptimizeProblem2(FIMSet):
    
    m = ConcreteModel()
    
    m.NoRes = Set(initialize=range(len(FIMSet)))
    m.DimFIM = Set(initialize=range(len(FIMSet[0][0])))
    
    m.y = Var(m.NoRes, initialize=0.1, bounds=(0,1), within = NonNegativeReals)

    def FIM_assign(m,y,d,e):
        return FIMSet[y][d,e]
    
    m.FIM = Param(m.NoRes, m.DimFIM, m.DimFIM, initialize=FIM_assign)
    
    def totalFIM(m,d,e):
        return sum(m.y[y] * m.FIM[y,d,e] for y in m.NoRes)
    
    m.TotalFIM = Var(m.DimFIM, m.DimFIM, initialize=totalFIM)
    
    def totalFIM_const(m,d,e):
        return m.TotalFIM[d,e] == sum(m.y[y] * m.FIM[y,d,e] for y in m.NoRes)
    m.TotalFIM_con = Constraint(m.DimFIM, m.DimFIM, rule=totalFIM_const)
    
    def ComputeTrace(m):
        sum_x = 0  
        for j in m.DimFIM:
            for d in m.DimFIM:
                if d==j:
                    sum_x += m.TotalFIM[j,d]
        return sum_x
    
    def det_general(m):
        '''Calculate determinant. Can be applied to FIM of any size.
        det(A) = sum_{\sigma \in \S_n} (sgn(\sigma) * \Prod_{i=1}^n a_{i,\sigma_i})
        Use permutation() to get permutations, sgn() to get signature
        '''
        r_list = list(range(4))
        # get all permutations
        object_p = permutations(r_list)
        list_p = list(object_p)

        # generate a name_order to iterate \sigma_i
        det_perm = 0
        for i in range(len(list_p)):
            name_order = []
            x_order = list_p[i]
            # sigma_i is the value in the i-th position after the reordering \sigma
            for x in range(len(x_order)):
                for y, element in enumerate(m.DimFIM):
                    if x_order[x] == y:
                        name_order.append(element)

        # det(A) = sum_{\sigma \in \S_n} (sgn(\sigma) * \Prod_{i=1}^n a_{i,\sigma_i})
        det_perm = sum(sgn(list_p[d])*sum(m.TotalFIM[each, name_order[b]] for b, each in enumerate(m.DimFIM)) for d in range(len(list_p)))
        return det_perm
    
    m.Obj = Objective(rule=det_general, sense=maximize)    
    
    def Sum1Const(m):
        return sum(m.y[y] for y in m.NoRes)<=20 
    m.sum1 = Constraint(rule=Sum1Const)
    
    return m 

In [ ]:
m = OptimizeProblem2(FIM_set)

solver = SolverFactory('ipopt')
solver.options['linear_solver']='ma57'

solver.solve(m, tee=True)

In [ ]:
for i in range(27):
    print(value(m.y[i]))

In [ ]:
print(value(m.Obj))
print(value(m.TotalFIM[0,0]))
print(value(m.TotalFIM[1,1]))
print(value(m.TotalFIM[2,2]))

print(value(m.TotalFIM[3,3]))

print(FIM_set[23])

print(sum(0.1*value(m.FIM[i,0,0]) for i in range(27)))